# DX 704 Week 4 Project

This week's project will test the learning speed of linear contextual bandits compared to unoptimized approaches.
You will start with building a preference data set for evaluation, and then implement different variations of LinUCB and visualize how fast they learn the preferences.


The full project description, a template notebook and supporting code are available on GitHub: [Project 4 Materials](https://github.com/bu-cds-dx704/dx704-project-04).


## Example Code

You may find it helpful to refer to these GitHub repositories of Jupyter notebooks for example code.

* https://github.com/bu-cds-omds/dx601-examples
* https://github.com/bu-cds-omds/dx602-examples
* https://github.com/bu-cds-omds/dx603-examples
* https://github.com/bu-cds-omds/dx704-examples

Any calculations demonstrated in code examples or videos may be found in these notebooks, and you are allowed to copy this example code in your homework answers.

## Part 1: Collect Rating Data

The file "recipes.tsv" in this repository has information about 100 recipes.
Make a new file "ratings.tsv" with two columns, recipe_slug (from recipes.tsv) and rating.
Populate the rating column with values between 0 and 1 where 0 is the worst and 1 is the best.
You can assign these ratings however you want within that range, but try to make it reflect a consistent set of preferences.
These could be your preferences, or a persona of your choosing (e.g. chocolate lover, bacon-obsessed, or sweet tooth).
Make sure that there are at least 10 ratings of zero and at least 10 ratings of one.


Hint: You may find it more convenient to assign raw ratings from 1 to 5 and then remap them as follows.

`ratings["rating"] = (ratings["rating_raw"] - 1) * 0.25`

In [1]:
import numpy as np
import pandas as pd

In [2]:
recipes = pd.read_csv('https://raw.githubusercontent.com/bu-cds-dx704/dx704-project-04/refs/heads/main/recipes.tsv', sep="\t")
recipes = recipes.set_index("recipe_slug")
recipes

,recipe_title,recipe_introduction
recipe_slug,,
falafel,Falafel,Falafel is a popular Middle Eastern dish made ...
spamburger,Spamburger,Spamburger is a type of hamburger that is made...
bacon-fried-rice,Bacon Fried Rice,Bacon fried rice is a savory and satisfying di...
chicken-fingers,Chicken Fingers,Chicken fingers are a popular dish made from c...
apple-crisp,Apple Crisp,Apple crisp is a classic dessert made with bak...
...,...,...
bacon-mac-and-cheese,Bacon Mac And Cheese,Bacon mac and cheese is a delicious and comfor...
chicken-alfredo-lasagna,Chicken Alfredo Lasagna,Chicken alfredo lasagna is a delicious twist o...
classic-beef-lasagna,Classic Beef Lasagna,Classic beef lasagna is a hearty and comfortin...


In [3]:
def add_feature(feature_name):
    recipes[feature_name] = recipes["recipe_title"].str.lower().str.contains(feature_name.lower())

In [4]:
add_feature("apple")
add_feature("bacon")
add_feature("chicken")
add_feature("crisp")
add_feature("lasagna")
add_feature("veg")

recipes

,recipe_title,recipe_introduction,apple,bacon,chicken,crisp,lasagna,veg
recipe_slug,,,,,,,,
falafel,Falafel,Falafel is a popular Middle Eastern dish made ...,False,False,False,False,False,False
spamburger,Spamburger,Spamburger is a type of hamburger that is made...,False,False,False,False,False,False
bacon-fried-rice,Bacon Fried Rice,Bacon fried rice is a savory and satisfying di...,False,True,False,False,False,False
chicken-fingers,Chicken Fingers,Chicken fingers are a popular dish made from c...,False,False,True,False,False,False
apple-crisp,Apple Crisp,Apple crisp is a classic dessert made with bak...,True,False,False,True,False,False
...,...,...,...,...,...,...,...,...
bacon-mac-and-cheese,Bacon Mac And Cheese,Bacon mac and cheese is a delicious and comfor...,False,True,False,False,False,False
chicken-alfredo-lasagna,Chicken Alfredo Lasagna,Chicken alfredo lasagna is a delicious twist o...,False,False,True,False,True,False
classic-beef-lasagna,Classic Beef Lasagna,Classic beef lasagna is a hearty and comfortin...,False,False,False,False,True,False


In [5]:
ratings = pd.DataFrame({"recipe_title": recipes["recipe_title"],
                               "rating_bad": np.random.uniform(low=2, high=3, size=len(recipes))},
                              index=recipes.index)
ratings.loc[recipes['veg'], 'rating_bad'] = 1
ratings.loc[recipes['bacon'], 'rating_bad'] = 5
ratings['rating'] = ratings['rating_bad']
ratings

,recipe_title,rating_bad,rating
recipe_slug,,,
falafel,Falafel,2.329163,2.329163
spamburger,Spamburger,2.476303,2.476303
bacon-fried-rice,Bacon Fried Rice,5.000000,5.000000
chicken-fingers,Chicken Fingers,2.634748,2.634748
apple-crisp,Apple Crisp,2.626110,2.626110
...,...,...,...
bacon-mac-and-cheese,Bacon Mac And Cheese,5.000000,5.000000
chicken-alfredo-lasagna,Chicken Alfredo Lasagna,2.044531,2.044531
classic-beef-lasagna,Classic Beef Lasagna,2.898128,2.898128


In [8]:
ratings["rating"] = (ratings["rating_bad"] - 1) * 0.25

In [ ]:
ratings

In [ ]:
ratings.to_csv("ratings.tsv", sep="\t", index=False)

Submit "ratings.tsv" in Gradescope.

## Part 2: Construct Model Input

Use your file "ratings.tsv" combined with "recipe-tags.tsv" to create a new file "features.tsv" with a column recipe_slug, a column bias which is hard-coded to one, and a column for each tag that appears in "recipe-tags.tsv".
The tag column in this file should be a 0-1 encoding of the recipe tags for each recipe.
[Pandas reshaping function methods](https://pandas.pydata.org/docs/user_guide/reshaping.html) may be helpful.

The bias column will make later LinUCB calculations easier since it will just be another dimension.

Hint: For later modeling steps, it will be important to have the feature data (inputs) and the rating data (target outputs) in the same order.
It is highly recommended to make sure that "features.tsv" and "ratings.tsv" have the recipe slugs in the same order.

In [11]:
# YOUR CHANGES HERE
features = recipes[[c for c in recipes.columns if not c.startswith("recipe_")]]
features

,apple,bacon,chicken,crisp,lasagna,veg
recipe_slug,,,,,,
falafel,False,False,False,False,False,False
spamburger,False,False,False,False,False,False
bacon-fried-rice,False,True,False,False,False,False
chicken-fingers,False,False,True,False,False,False
apple-crisp,True,False,False,True,False,False
...,...,...,...,...,...,...
bacon-mac-and-cheese,False,True,False,False,False,False
chicken-alfredo-lasagna,False,False,True,False,True,False
classic-beef-lasagna,False,False,False,False,True,False


In [15]:
features = features.astype({c: int for c in features.select_dtypes(bool).columns})
features

,apple,bacon,chicken,crisp,lasagna,veg
recipe_slug,,,,,,
falafel,0,0,0,0,0,0
spamburger,0,0,0,0,0,0
bacon-fried-rice,0,1,0,0,0,0
chicken-fingers,0,0,1,0,0,0
apple-crisp,1,0,0,1,0,0
...,...,...,...,...,...,...
bacon-mac-and-cheese,0,1,0,0,0,0
chicken-alfredo-lasagna,0,0,1,0,1,0
classic-beef-lasagna,0,0,0,0,1,0


In [16]:
features.sum()

apple       4
bacon      13
chicken     5
crisp       6
lasagna     6
veg         2
dtype: int64

In [18]:
features.T.sum().T

recipe_slug
falafel                        0
spamburger                     0
bacon-fried-rice               1
chicken-fingers                1
apple-crisp                    2
                              ..
bacon-mac-and-cheese           1
chicken-alfredo-lasagna        2
classic-beef-lasagna           1
vegetarian-mushroom-lasagna    2
spinach-and-ricotta-lasagna    1
Length: 100, dtype: int64

In [19]:
features.to_csv("features.tsv", sep="\t", index=False)

Submit "features.tsv" in Gradescope.

## Part 3: Linear Preference Model

Use your feature and rating files to build a ridge regression model with ridge regression's regularization parameter $\alpha$ set to 1.


Hint: If you are using scikit-learn modeling classes, you should use `fit_intercept=False` since that intercept value will be redundant with the bias coefficient.

Hint: The estimate component of the bounds should match the previous estimate, so you should be able to just focus on the variance component of the bounds now.

In [ ]:
# YOUR CHANGES HERE

...

Save the coefficients of this model in a file "model.tsv" with columns "recipe_tag" and "coefficient".
Do not include the bias.

In [ ]:
# YOUR CHANGES HERE

...

Submit "model.tsv" in Gradescope.

## Part 4: Recipe Estimates

Use the recipe model to estimate the score of every recipe.
Save these estimates to a file "estimates.tsv" with columns recipe_slug and score_estimate.

In [ ]:
# YOUR CHANGES HERE

...

Submit "estimates.tsv" in Gradescope.

## Part 5: LinUCB Bounds

Calculate the upper bounds of LinUCB using data corresponding to trying every recipe once and receiving the rating in "ratings.tsv" as the reward.
Keep the ridge regression regularization parameter at 1, and set LinUCB's $\alpha$ parameter to 2.
Save these upper bounds to a file "bounds.tsv" with columns recipe_slug and score_bound.

In [ ]:
# YOUR CHANGES HERE

def calculate_bounds(recipe_choices=[], alpha=1.0):
    recipe_choices = list(recipe_choices)

    D = features.loc[recipe_choices].to_numpy(dtype="float64")
    c = ratings.loc[recipe_choices, 'rating'].to_numpy(dtype="float64")

    DTDI = np.eye(D.shape[1])
    if len(recipe_choices) > 0:
        DTDI += D.T @ D
    DTDI_inv = np.linalg.inv(DTDI)

    theta_hat = DTDI_inv @ D.T @ c
    features_array = features.to_numpy()
    means = features_array @ theta_hat

    variances = []
    for z in features_array:
        z = z.reshape(-1, 1)
        variances.append((z.T @ DTDI_inv @ z).item())
    variances = np.array(variances)

    df = pd.DataFrame({"score_estimate": means,
    "score_bound": means + alpha * np.sqrt(variances)}, index=features.index)
    df["num_features"] = features.T.sum().T

    return df

calculate_bounds().sort_values("score_bound", ascending=False)

In [ ]:
calculate_bounds(["bacon-mac-and-cheese"]* 3)

Submit "bounds.tsv" in Gradescope.

## Part 6: Make Online Recommendations

Implement LinUCB to make 100 recommendations starting with no data and using the same parameters as in part 5.
One recommendation should be made at a time and you can break ties arbitrarily.
After each recommendation, use the rating from part 1 as the reward to update the LinUCB data.
Record the recommendations made in a file "recommendations.tsv" with columns "recipe_slug", "score_bound", and "reward".
The rows in this file should be in the same order as the recommendations were made.

In [ ]:
# YOUR CHANGES HERE
def try_picks(**kwargs):
    recipe_choices = []
    for i in range(100):
        current_bounds = calculate_bounds([r.index[0] for r in recipe_choices], **kwargs)
        best_bound = current_bounds["score_bound"].max()
        best_recipes = current_bounds[current_bounds["score_bound"] == best_bound].copy()
        best_recipes["true_rating"] = ratings.loc[best_recipes.index, "rating"]
        choice = best_recipes.sample(1)
        recipe_choices.append(choice)

    return pd.concat(recipe_choices, axis=0).reset_index()

picks = try_picks()
picks

Submit "recommendations.tsv" in Gradescope.

## Part 7: Acknowledgments

Make a file "acknowledgments.txt" documenting any outside sources or help on this project.
If you discussed this assignment with anyone, please acknowledge them here.
If you used any libraries not mentioned in this module's content, please list them with a brief explanation what you used them for.
If you used any generative AI tools, please add links to your transcripts below, and any other information that you feel is necessary to comply with the generative AI policy.
If no acknowledgements are appropriate, just write none in the file.


Submit "acknowledgments.txt" in Gradescope.

## Part 8: Code

Please submit a Jupyter notebook that can reproduce all your calculations and recreate the previously submitted files.


Submit "project.ipynb" in Gradescope.